# Evaluation of Model and ECE/MCE

## Stochastic Depth

In [1]:
# Used code from https://github.com/preddy5/Residual-Learning-and-Stochastic-Depth as base
# coding: utf-8

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu1,floatX=float32,optimizer=None'
import numpy as np
import collections
import pickle

from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.optimizers import SGD
import keras.backend as K
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model
from keras.callbacks import (
    Callback,
    LearningRateScheduler,
)
from keras.layers import (
    Input,
    Activation,
    Add,
    Dense,
    Flatten,
    Lambda
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from resnet import (
    bn_relu_conv,
    conv_bn_relu,
    residual_block
)

from calibration import evaluate_model

Using TensorFlow backend.


In [2]:
def _bottleneck(input, nb_filters, init_subsample=(1, 1)):
    conv_1_1 = bn_relu_conv(input, nb_filters, 3, 3, W_regularizer=l2(weight_decay), subsample=init_subsample)
    conv_3_3 = bn_relu_conv(conv_1_1, nb_filters, 3, 3, W_regularizer=l2(weight_decay))
    return _shortcut(input, conv_3_3)

    
def _shortcut(input, residual):
    stride_width = input._keras_shape[2] // residual._keras_shape[2]
    stride_height = input._keras_shape[3] // residual._keras_shape[3]
    equal_channels = residual._keras_shape[1] == input._keras_shape[1]

    shortcut = input
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual._keras_shape[1], kernel_size=(1, 1), strides=(stride_width, stride_height),
                         kernel_initializer="he_normal", padding="valid",kernel_regularizer=l2(weight_decay),
                                 data_format="channels_first")(input)
        shortcut = Activation("relu")(shortcut)

    M1 = Add()([shortcut, residual])
    M1 = Activation("relu")(M1)
    
    gate = K.variable(0, dtype="int16")
    decay_rate = 1
    name = 'residual_'+str(len(gates)+1)
    gates[name]=[decay_rate, gate]
    return Lambda(lambda outputs: K.switch(gate, outputs[0], outputs[1]),
                  output_shape= lambda x: x[0], name=name)([shortcut, M1])


# http://arxiv.org/pdf/1512.03385v1.pdf
# 110 Layer resnet
    # repetations: = n*6 + 2 = 18*6 + 2 = 110
    # nr_classes: int - how many classes in the end
def resnet(n = 18, nr_classes=10):
    input = Input(shape=(img_channels, img_rows, img_cols))

    conv1 = conv_bn_relu(input, nb_filter=16, nb_row=3, nb_col=3, W_regularizer=l2(weight_decay))  # Filters, filter_size

    # Build residual blocks..
    block_fn = _bottleneck
    block1 = residual_block(conv1, block_fn, nb_filters=16, repetations=n, is_first_layer=True)
    block2 = residual_block(block1, block_fn, nb_filters=32, repetations=n)
    block3 = residual_block(block2, block_fn, nb_filters=64, repetations=n, subsample=True)
    
    # Classifier block
    pool2 = AveragePooling2D(pool_size=(8, 8))(block3)
    flatten1 = Flatten()(pool2)
    final = Dense(units=nr_classes, kernel_initializer="he_normal", activation="softmax", kernel_regularizer=l2(weight_decay))(flatten1)

    model = Model(inputs=input, outputs=final)
    return model



def set_decay_rate():
    for index, key in enumerate(gates):
        gates[key][0] = 1.0 - float(index)*pL / len(gates)

# Callbacks for updating gates and learning rate
def scheduler(epoch):

    if epoch < 2:
        return learning_rate*0.1
    elif epoch < nb_epochs/2:
        return learning_rate
    elif epoch < nb_epochs*3/4:
        return learning_rate*0.1
    return learning_rate*0.01


class Gates_Callback(Callback):
    def on_batch_begin(self, batch, logs={}):
        probs = np.random.uniform(size=len(gates))
        for i,j in zip(gates, probs):
            if j > gates[i][0]:
                K.set_value(gates[i][1], 1)
            else:
                K.set_value(gates[i][1], 0)

    def on_train_end(self, logs={}):
        for i in gates:
            K.set_value(gates[i][1],1)

In [3]:
# constants
learning_rate = 0.1
momentum = 0.9
img_rows, img_cols = 32, 32
img_channels = 3
nb_epochs = 500
batch_size = 128
nb_classes = 10
nb_calsses100 = 100
pL = 1  # For testing mode
weight_decay = 1e-4
seed = 333


# data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = np.transpose(x_train.astype('float32'), (0, 3, 1, 2))  # Channels first
x_test = np.transpose(x_test.astype('float32'), (0, 3, 1, 2))  # Channels first


# Data splitting (get additional 5k validation set)
# Sklearn to split
x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed

img_mean = x_train45.mean(axis=0)  # per-pixel mean
img_std = x_train45.std(axis=0)
x_train45 = (x_train45-img_mean)/img_std
x_val = (x_val-img_mean)/img_std
x_test = (x_test-img_mean)/img_std


img_gen = ImageDataGenerator(
    horizontal_flip=True,
    data_format="channels_first",
    width_shift_range=0.125,  # 0.125*32 = 4 so max padding of 4 pixels, as described in paper.
    height_shift_range=0.125,
    fill_mode="constant",
    cval = 0
)

img_gen.fit(x_train45)
y_train45 = np_utils.to_categorical(y_train45, nb_classes)  # 1-hot vector
y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
    

### Load in data and models

In [4]:
        
# building and training net
gates=collections.OrderedDict()
model = resnet(nr_classes=nb_classes)
#set_decay_rate()
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="categorical_crossentropy",metrics=["accuracy"])  

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 32, 32)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 16, 32, 32)    448         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 16, 32, 32)    64          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 16, 32, 32)    0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [5]:
for i in gates:
    print(K.get_value(gates[i][1]), gates[i][0],i)

0 1 residual_1
0 1 residual_2
0 1 residual_3
0 1 residual_4
0 1 residual_5
0 1 residual_6
0 1 residual_7
0 1 residual_8
0 1 residual_9
0 1 residual_10
0 1 residual_11
0 1 residual_12
0 1 residual_13
0 1 residual_14
0 1 residual_15
0 1 residual_16
0 1 residual_17
0 1 residual_18
0 1 residual_19
0 1 residual_20
0 1 residual_21
0 1 residual_22
0 1 residual_23
0 1 residual_24
0 1 residual_25
0 1 residual_26
0 1 residual_27
0 1 residual_28
0 1 residual_29
0 1 residual_30
0 1 residual_31
0 1 residual_32
0 1 residual_33
0 1 residual_34
0 1 residual_35
0 1 residual_36
0 1 residual_37
0 1 residual_38
0 1 residual_39
0 1 residual_40
0 1 residual_41
0 1 residual_42
0 1 residual_43
0 1 residual_44
0 1 residual_45
0 1 residual_46
0 1 residual_47
0 1 residual_48
0 1 residual_49
0 1 residual_50
0 1 residual_51
0 1 residual_52
0 1 residual_53
0 1 residual_54


In [ ]:
from utils import calibration


In [6]:
import sklearn.metrics as metrics

def evaluate_model(model, weights_file, x_test, y_test, bins = 15, verbose = True):
    """
    Evaluates the model, in addition calculates the calibration errors
    
    Args:
        model (keras.model): constructed model
        weights (string): path to weights file
        x_test: (np.array) with x_test data (already in right format)
        y_test: (np.array) with y_test data (1-hot vector)
        
    Returns:
        (acc, ece, mce): accuracy of model, ECE and MCE (calibration errors)
    """
    
    # First load in the weights
    model.load_weights(weights_file, by_name=True)
    
    # Next get predictions
    y_probs = model.predict(x_test)
    y_preds = np.argmax(y_probs, axis=1)
    y_true = y_test
    
    # Find accuracy and error
    y_true = [ np.where(r==1)[0][0] for r in y_true]  # 1-hot vector back to numeric
    accuracy = metrics.accuracy_score(y_true, y_preds) * 100
    error = 100 - accuracy
    
    # Confidence of prediction
    y_confs = np.max(y_probs, axis=1)  # Take only maximum confidence
    
    # Calculate ECE
    ece = ECE(y_confs, y_preds, y_true, bin_size = 1/bins)
    # Calculate MCE
    mce = MCE(y_confs, y_preds, y_true, bin_size = 1/bins)
    
    if verbose:
        print("Accuracy:", accuracy)
        print("Error:", error)
        print("ECE:", ece)
        print("MCE:", mce)
    
    return (accuracy, ece, mce)    

### CIFAR-10

### Load in weights

In [7]:
weights_file = "../models/model_weight_ep500_110SD_cifar_10.hdf5"
# set optimizer
sgd = SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.load_weights(weights_file, by_name=True)

### Get predictions

In [8]:
y_probs = model.predict(x_test)
y_preds = np.argmax(y_probs, axis=1)
y_true = y_test

In [9]:
import sklearn.metrics as metrics
y_true = [ np.where(r==1)[0][0] for r in y_true]  # 1-hot vector back to numeric
accuracy = metrics.accuracy_score(y_true, y_preds) * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)

Accuracy :  72.6
Error :  27.4


In [10]:
y_probs[4]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.52645253e-31,   0.00000000e+00,
         1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00], dtype=float32)

### Get confidences

In [19]:
y_confs = np.max(y_probs, axis=1)

In [20]:
y_confs[:5]

array([ 1.        ,  0.99999952,  0.98878014,  0.59405005,  1.        ], dtype=float32)

In [28]:
def compute_acc_bin(conf_thresh_lower, conf_thresh_upper, conf, pred, true):
    """
    # Computes accuracy and average confidence for bin
    
    Args:
        conf_thresh_lower (float): Lower Threshold of confidence interval
        conf_thresh_upper (float): Upper Threshold of confidence interval
        conf (np.array): list of confidences
        pred (np.array): list of predictions
        true (np.array): list of true labels
    
    Returns:
        (accuracy, avg_conf, len_bin): accuracy of bin, confidence of bin and number of elements in bin.
    """
    filtered_tuples = [x for x in zip(pred, true, conf) if x[2] > conf_thresh_lower and x[2] <= conf_thresh_upper]
    if len(filtered_tuples) < 1:
        return 0,0,0
    else:
        correct = len([x for x in filtered_tuples if x[0] == x[1]])  # How many correct labels
        len_bin = len(filtered_tuples)  # How many elements falls into given bin
        avg_conf = sum([x[2] for x in filtered_tuples]) / len_bin  # Avg confidence of BIN
        accuracy = float(correct)/len_bin  # accuracy of BIN
        return accuracy, avg_conf, len_bin
    
    
def ECE(conf, pred, true, bin_size = 0.1):

    """
    Expected Calibration Error
    
    Args:
        conf (np.array): list of confidences
        pred (np.array): list of predictions
        true (np.array): list of true labels
        bin_size: (float): size of one bin (0,1)  # TODO should convert to number of bins?
        
    Returns:
        ece: expected calibration error
    """
    
    upper_bounds = np.arange(bin_size, 1+bin_size, bin_size)  # Get bounds of bins
    
    n = len(conf)
    ece = 0  # Starting error
    
    for conf_thresh in upper_bounds:  # Go through bounds and find accuracies and confidences
        acc, avg_conf, len_bin = compute_acc_bin(conf_thresh-bin_size, conf_thresh, conf, pred, true)        
        ece += np.abs(acc-avg_conf)*len_bin/n  # Add weigthed difference to ECE
        
    return ece
        
      
def MCE(conf, pred, true, bin_size = 0.1):

    """
    Maximal Calibration Error
    
    Args:
        conf (np.array): list of confidences
        pred (np.array): list of predictions
        true (np.array): list of true labels
        bin_size: (float): size of one bin (0,1)  # TODO should convert to number of bins?
        
    Returns:
        mce: maximum calibration error
    """
    
    upper_bounds = np.arange(bin_size, 1+bin_size, bin_size)
    
    cal_errors = []
    
    for conf_thresh in upper_bounds:
        acc, avg_conf, _ = compute_acc_bin(conf_thresh-bin_size, conf_thresh, conf, pred, true)
        cal_errors.append(np.abs(acc-avg_conf))
        
    return max(cal_errors)

In [29]:
ECE(y_confs, y_preds, y_true)

0.22742563225179907

In [30]:
MCE(y_confs, y_preds, y_true)

0.41998978786151425

In [31]:
type(y_confs)

numpy.ndarray